In [ ]:
from keras.layers import Input, Dense, Activation, Cropping2D, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Concatenate, Add
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, UpSampling2D
from keras.models import Model
from keras.utils import layer_utils, plot_model, to_categorical, multi_gpu_model, normalize
import keras.backend as K
from PIL import Image
import numpy as np
import keras
#import models
from matplotlib import pyplot as plt
from keras.optimizers import Adam, rmsprop
import math
from keras.callbacks import TensorBoard, ModelCheckpoint, Callback, ReduceLROnPlateau, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import cv2

#Import dataset

importdataset=np.load("Numpy arrays/images.npy")
importlabels=to_categorical(np.load("Numpy arrays/masks.npy"))

print(importdataset.shape)
print(importdataset.dtype)
print(importlabels.shape)

print('Done!')

In [ ]:
#Split each image into multiple smaller images

xsplit=1
ysplit=1

dataset=np.zeros((importdataset.shape[0]*xsplit*ysplit,int(importdataset.shape[1]/ysplit),int(importdataset.shape[2]/xsplit),importdataset.shape[3]), dtype = 'float32')
labels=np.zeros((importlabels.shape[0]*xsplit*ysplit,int(importlabels.shape[1]/ysplit),int(importlabels.shape[2]/xsplit),importlabels.shape[3]), dtype = 'bool')


for i in range(xsplit*ysplit):
    print(i+1,'out of',xsplit*ysplit)
    dataset[i*importdataset.shape[0]:(i+1)*importdataset.shape[0],:,:,:]=importdataset[:,i%ysplit*int(importdataset.shape[1]/ysplit):(i%ysplit+1)*int(importdataset.shape[1]/ysplit),math.floor(i/ysplit)%xsplit*int(importdataset.shape[2]/xsplit):(math.floor(i/ysplit)%xsplit+1)*int(importdataset.shape[2]/xsplit),:].astype('float32')/255
    labels[i*importlabels.shape[0]:(i+1)*importlabels.shape[0],:,:,:]=importlabels[:,i%ysplit*int(importlabels.shape[1]/ysplit):(i%ysplit+1)*int(importlabels.shape[1]/ysplit),math.floor(i/ysplit)%xsplit*int(importlabels.shape[2]/xsplit):(math.floor(i/ysplit)%xsplit+1)*int(importlabels.shape[2]/xsplit),:].astype('bool')

NoOfImages=dataset.shape[0]

#x_train=dataset[:int(NoOfImages*2/3),:,:,:]
#x_test=dataset[int(NoOfImages*2/3):NoOfImages,:,:,:]
#y_train=to_categorical(labels[:int(NoOfImages*2/3),:,:])
#y_test=to_categorical(labels[int(NoOfImages*2/3):NoOfImages,:,:])
#importdataset
#importlabels
print('Done!')

In [ ]:
del importdataset

#Resize images to 224x224

resized_dataset=np.zeros((dataset.shape[0],224,224,3), dtype='float32')
resized_labels=np.zeros((labels.shape[0],224,224,2), dtype='float32')
for i in range(dataset.shape[0]):
    resized_dataset[i] = cv2.resize(dataset[i,...], dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
    resized_labels[i] = cv2.resize(labels[i,...].astype('float32'), dsize=(224, 224), interpolation=cv2.INTER_CUBIC).astype('bool')

del dataset
print(np.amax(resized_dataset))
#dataset=2*dataset[labels[..., 1].any(axis=((1,2)))].astype('float32')-1
dataset=dataset[labels[..., 1].any(axis=((1,2)))].astype('float32')
labels=labels[labels[..., 1].any(axis=((1,2)))]
resized_dataset=2*resized_dataset-1
#dataset=dataset[labels[..., 1].any(axis=((1,2)))].astype('float32')
print(np.amax(resized_dataset))
    
    

In [ ]:
#sanity check
print(resized_dataset.dtype)
print(resized_labels.dtype)
print(resized_dataset.shape)
print(resized_labels.shape)
a=np.sum(resized_labels[...,0])
b=np.sum(resized_labels[...,1])
print(a)
print(b)
print(a/(b+a))
print(resized_dataset[68,...])

In [ ]:
#sanity check

f = plt.figure()
image=109
f.add_subplot(1,2, 1)
#plt.imshow(resized_dataset[image,:,:,:].astype('float32')/2+[.5])
f.add_subplot(1,2, 2)
plt.imshow(resized_labels[image,:,:,1])
plt.show(block=True)

In [ ]:
#Resunet with variable parameter number

K.set_image_data_format('channels_last')
K.set_floatx('float32')
smallest_layer=64

def RESUNET(input_shape, n_classes):

    inputs = Input(input_shape)

    ###encoding block 1
    iden1 = Conv2D(smallest_layer, 1, activation = None, padding='same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(smallest_layer, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(inputs) #200
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = Conv2D(smallest_layer, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv1) #200
    add1 = Add()([iden1,conv1])
    pool1 = MaxPooling2D()(add1) #200 -> 100
    print(add1)

    ###encoding block2
    iden2 = Conv2D(smallest_layer*2, 1, activation = None, padding='same', kernel_initializer = 'he_normal')(pool1)
    conv2 = BatchNormalization()(pool1)
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(smallest_layer*2, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv2) #200
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(smallest_layer*2, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv2) #200
    add2 = Add()([iden2,conv2])
    pool2 = MaxPooling2D()(add2) #100 ->50
    print (add2)

    ###encoding block3
    iden3 = Conv2D(smallest_layer*4, 1, activation = None, padding='same', kernel_initializer = 'he_normal')(pool2)
    conv3 = BatchNormalization()(pool2)
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv3) #200
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv3) #200
    add3 = Add()([iden3,conv3])
    pool3= MaxPooling2D()(add3) #50->25
    print (add3)

    ###encoding block4
    iden4 = Conv2D(smallest_layer*8, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(pool3)
    conv4 = BatchNormalization()(pool3)
    conv4 = Activation('relu')(conv4)
    conv4 = Conv2D(smallest_layer*8, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv4) #200
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    conv4 = Conv2D(smallest_layer*8, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv4) #200
    add4 = Add()([iden4,conv4])
    drop4 = Dropout(0.5)(add4)
    pool4 = MaxPooling2D()(drop4) #25->12
    print (pool4)

    ###bridge
    conv5 = BatchNormalization()(pool4)
    conv5 = Activation('relu')(conv5)
    conv5 = Conv2D(smallest_layer*16, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv5) #200
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)
    conv5 = Conv2D(smallest_layer*16, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv5) #200
    drop5 = Dropout(0.5)(conv5)
    print (conv5)

    ###decoding block1
    up6 = UpSampling2D()(drop5) #12->24
    up6 = ZeroPadding2D(((1,0),(1,0)))(up6) #24->25
    concat6 = Concatenate(axis=3)([up6,add4])
    iden6 = Conv2D(smallest_layer*8, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(concat6)
    conv6 = BatchNormalization()(concat6)
    conv6 = Activation('relu')(conv6)
    conv6 = Conv2D(smallest_layer*8, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv6) #200
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation('relu')(conv6)
    conv6 = Conv2D(smallest_layer*8, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv6) #200
    add6 = Add()([iden6,conv6])

    ###decoding block2
    up7 = UpSampling2D()(add6) #25->50
    up7 = ZeroPadding2D(((0,0),(0,0)))(up7) #24->25
    concat7 = Concatenate(axis=3)([up7,add3])
    iden7 = Conv2D(smallest_layer*4, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(concat7)
    conv7 = BatchNormalization()(concat7)
    conv7 = Activation('relu')(conv7)
    conv7 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv7) #200
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation('relu')(conv7)
    conv7 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv7) #200
    add7 = Add()([iden7,conv7])

    ###decoding block3
    up8 = UpSampling2D()(add7) #50->100
    up8 = ZeroPadding2D(((0,0),(0,0)))(up8) #24->25
    concat8 = Concatenate(axis=3)([up8,add2])
    iden8 = Conv2D(smallest_layer*2, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(concat8)
    conv8 = BatchNormalization()(concat8)
    conv8 = Activation('relu')(conv8)
    conv8 = Conv2D(smallest_layer*2, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv8) #200
    conv8 = BatchNormalization()(conv8)
    conv8 = Activation('relu')(conv8)
    conv8 = Conv2D(smallest_layer*2, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv8) #200
    add8 = Add()([iden8,conv8])

    ###decoding block4
    up9 = UpSampling2D()(add8) #100->200
    up9 = ZeroPadding2D(((0,0),(0,0)))(up9) #24->25
    concat9 = Concatenate(axis=3)([up9,add1])
    iden9 = Conv2D(smallest_layer,1,activation=None, padding='same', kernel_initializer = 'he_normal')(concat9)
    conv9 = BatchNormalization()(concat9)
    conv9 = Activation('relu')(conv9)
    conv9 = Conv2D(smallest_layer, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv9) #200
    conv9 = BatchNormalization()(conv9)
    conv9 = Activation('relu')(conv9)
    conv9 = Conv2D(smallest_layer, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv9) #200
    add9 = Add()([iden9,conv9])

    conv10 = Conv2D(n_classes, 3, activation ='sigmoid', padding = 'same', kernel_initializer = 'he_normal')(add9)
    #sigmoid probably too strong an activation

    model = Model(input = inputs, output = conv10)

    return model


In [ ]:
#Resunet with less layers

K.set_image_data_format('channels_last')
K.set_floatx('float32')
smallest_layer=32

def RESUNETMEDIUM(input_shape, n_classes):

    inputs = Input(input_shape)

    ###encoding block 1
    iden1 = Conv2D(smallest_layer, 1, activation = None, padding='same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(smallest_layer, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(inputs) #200
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = Conv2D(smallest_layer, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv1) #200
    add1 = Add()([iden1,conv1])
    pool1 = MaxPooling2D((3,3))(add1) #224 -> 74
    print(add1)

    ###encoding block2
    iden2 = Conv2D(smallest_layer*2, 1, activation = None, padding='same', kernel_initializer = 'he_normal')(pool1)
    conv2 = BatchNormalization()(pool1)
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(smallest_layer*2, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv2) #200
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(smallest_layer*2, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv2) #200
    add2 = Add()([iden2,conv2])
    pool2 = MaxPooling2D((3,3))(add2) #74 -> 24
    print (add2)

    ###encoding block4
    iden4 = Conv2D(smallest_layer*8, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(pool2)
    conv4 = BatchNormalization()(pool2)
    conv4 = Activation('relu')(conv4)
    conv4 = Conv2D(smallest_layer*8, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv4) #200
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    conv4 = Conv2D(smallest_layer*8, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv4) #200
    add4 = Add()([iden4,conv4])
    drop4 = Dropout(0.5)(add4)
    pool4 = MaxPooling2D((3,3))(drop4) #24 -> 8
    print (pool4)

    ###bridge
    conv5 = BatchNormalization()(pool4)
    conv5 = Activation('relu')(conv5)
    conv5 = Conv2D(smallest_layer*16, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv5) #200
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)
    conv5 = Conv2D(smallest_layer*16, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv5) #200
    drop5 = Dropout(0.5)(conv5)
    print (conv5)

    ###decoding block1
    up6 = UpSampling2D((3,3))(drop5) #8->24
    #up6 = ZeroPadding2D(((1,0),(1,0)))(up6) #24->25
    concat6 = Concatenate(axis=3)([up6,add4])
    iden6 = Conv2D(smallest_layer*8, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(concat6)
    conv6 = BatchNormalization()(concat6)
    conv6 = Activation('relu')(conv6)
    conv6 = Conv2D(smallest_layer*8, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv6) #200
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation('relu')(conv6)
    conv6 = Conv2D(smallest_layer*8, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv6) #200
    add6 = Add()([iden6,conv6])

    ###decoding block3
    up8 = UpSampling2D((3,3))(add6) #24 -> 72
    up8 = ZeroPadding2D(((0,2),(0,2)))(up8) #72->74
    concat8 = Concatenate(axis=3)([up8,add2])
    iden8 = Conv2D(smallest_layer*2, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(concat8)
    conv8 = BatchNormalization()(concat8)
    conv8 = Activation('relu')(conv8)
    conv8 = Conv2D(smallest_layer*2, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv8) #200
    conv8 = BatchNormalization()(conv8)
    conv8 = Activation('relu')(conv8)
    conv8 = Conv2D(smallest_layer*2, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv8) #200
    add8 = Add()([iden8,conv8])

    ###decoding block4
    up9 = UpSampling2D((3,3))(add8) #74 -> 222
    up9 = ZeroPadding2D(((0,2),(0,2)))(up9) #222->224
    concat9 = Concatenate(axis=3)([up9,add1])
    iden9 = Conv2D(smallest_layer,1,activation=None, padding='same', kernel_initializer = 'he_normal')(concat9)
    conv9 = BatchNormalization()(concat9)
    conv9 = Activation('relu')(conv9)
    conv9 = Conv2D(smallest_layer, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv9) #200
    conv9 = BatchNormalization()(conv9)
    conv9 = Activation('relu')(conv9)
    conv9 = Conv2D(smallest_layer, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv9) #200
    add9 = Add()([iden9,conv9])

    conv10 = Conv2D(n_classes, 3, activation ='sigmoid', padding = 'same', kernel_initializer = 'he_normal')(add9)
    #sigmoid probably too strong an activation

    model = Model(input = inputs, output = conv10)

    return model

    



In [ ]:
#Resunet with even less layers
K.set_image_data_format('channels_last')
K.set_floatx('float32')
smallest_layer=42

def RESUNETSMALL(input_shape, n_classes):
    inputs = Input(input_shape)

    ###encoding block 1
    iden1 = Conv2D(smallest_layer, 1, activation = None, padding='same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(smallest_layer, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(inputs) #200
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = Conv2D(smallest_layer, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv1) #200
    add1 = Add()([iden1,conv1])
    pool1 = MaxPooling2D((4,4))(add1) #224 -> 56
    print(add1)

    ###encoding block3
    iden3 = Conv2D(smallest_layer*4, 1, activation = None, padding='same', kernel_initializer = 'he_normal')(pool1)
    conv3 = BatchNormalization()(pool1)
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv3) #200
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv3) #200
    add3 = Add()([iden3,conv3])
    #drop3 = Dropout(0.5)(add3)
    pool3= MaxPooling2D((4,4))(add3)#(drop3) #56 -> 14
    print (add3)

    ###bridge
    conv5 = BatchNormalization()(pool3)
    conv5 = Activation('relu')(conv5)
    conv5 = Conv2D(smallest_layer*16, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv5) #200
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)
    conv5 = Conv2D(smallest_layer*16, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv5) #200
    drop5 = Dropout(0.5)(conv5)
    print (conv5)

    ###decoding block2
    up7 = UpSampling2D((4,4))(drop5) #40-> 160
    up7 = ZeroPadding2D(((0,0),(0,0)))(up7) #160->162
    concat7 = Concatenate(axis=3)([up7,add3])
    iden7 = Conv2D(smallest_layer*4, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(concat7)
    conv7 = BatchNormalization()(concat7)
    conv7 = Activation('relu')(conv7)
    conv7 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv7) #200
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation('relu')(conv7)
    conv7 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv7) #200
    add7 = Add()([iden7,conv7])

    ###decoding block4
    up9 = UpSampling2D((4,4))(add7) #162->648
    up9 = ZeroPadding2D(((0,0),(0,0)))(up9) #24->25
    concat9 = Concatenate(axis=3)([up9,add1])
    iden9 = Conv2D(smallest_layer,1,activation=None, padding='same', kernel_initializer = 'he_normal')(concat9)
    conv9 = BatchNormalization()(concat9)
    conv9 = Activation('relu')(conv9)
    conv9 = Conv2D(smallest_layer, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv9) #200
    conv9 = BatchNormalization()(conv9)
    conv9 = Activation('relu')(conv9)
    conv9 = Conv2D(smallest_layer, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv9) #200
    add9 = Add()([iden9,conv9])

    conv10 = Conv2D(n_classes, 3, activation ='sigmoid', padding = 'same', kernel_initializer = 'he_normal')(add9)
    #sigmoid probably too strong an activation

    model = Model(input = inputs, output = conv10)

    return model

In [ ]:
#modified Resnet152
import cv2
import numpy as np
import copy

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Flatten, Activation, add
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import initializers
from keras.engine import Layer, InputSpec
from keras import backend as K

import sys
sys.setrecursionlimit(3000)

trainable = 'False'

class Scale(Layer):
    '''Custom Layer for ResNet used for BatchNormalization.
    
    Learns a set of weights and biases used for scaling the input data.
    the output consists simply in an element-wise multiplication of the input
    and a sum of a set of constants:
        out = in * gamma + beta,
    where 'gamma' and 'beta' are the weights and biases larned.
    # Arguments
        axis: integer, axis along which to normalize in mode 0. For instance,
            if your input tensor has shape (samples, channels, rows, cols),
            set axis to 1 to normalize per feature map (channels axis).
        momentum: momentum in the computation of the
            exponential average of the mean and standard deviation
            of the data, for feature-wise normalization.
        weights: Initialization weights.
            List of 2 Numpy arrays, with shapes:
            `[(input_shape,), (input_shape,)]`
        beta_init: name of initialization function for shift parameter
            (see [initializers](../initializers.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
        gamma_init: name of initialization function for scale parameter (see
            [initializers](../initializers.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
    '''
    def __init__(self, weights=None, axis=-1, momentum = 0.9, beta_init='zero', gamma_init='one', **kwargs):
        self.momentum = momentum
        self.axis = axis
        self.beta_init = initializers.get(beta_init)
        self.gamma_init = initializers.get(gamma_init)
        self.initial_weights = weights
        super(Scale, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        shape = (int(input_shape[self.axis]),)

        self.gamma = K.variable(self.gamma_init(shape), name='%s_gamma'%self.name)
        self.beta = K.variable(self.beta_init(shape), name='%s_beta'%self.name)
        self.trainable_weights = [self.gamma, self.beta]

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights

    def call(self, x, mask=None):
        input_shape = self.input_spec[0].shape
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]

        out = K.reshape(self.gamma, broadcast_shape) * x + K.reshape(self.beta, broadcast_shape)
        return out

    def get_config(self):
        config = {"momentum": self.momentum, "axis": self.axis}
        base_config = super(Scale, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

def identity_block(input_tensor, kernel_size, filters, stage, block):
    '''The identity_block is the block that has no conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    '''
    eps = 1.1e-5
    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    x = Conv2D(nb_filter1, (1, 1), name=conv_name_base + '2a', use_bias=False, trainable=trainable)(input_tensor)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2a')(x)
    x = Activation('relu', name=conv_name_base + '2a_relu')(x)

    x = ZeroPadding2D((1, 1), name=conv_name_base + '2b_zeropadding')(x)
    x = Conv2D(nb_filter2, (kernel_size, kernel_size), name=conv_name_base + '2b', use_bias=False, trainable=trainable)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2b')(x)
    x = Activation('relu', name=conv_name_base + '2b_relu')(x)

    x = Conv2D(nb_filter3, (1, 1), name=conv_name_base + '2c', use_bias=False, trainable=trainable)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2c')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2c')(x)

    x = add([x, input_tensor], name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x

def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    '''conv_block is the block that has a conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    Note that from stage 3, the first conv layer at main path is with subsample=(2,2)
    And the shortcut should have subsample=(2,2) as well
    '''
    eps = 1.1e-5
    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    x = Conv2D(nb_filter1, (1, 1), strides=strides, name=conv_name_base + '2a', use_bias=False, trainable=trainable)(input_tensor)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2a')(x)
    x = Activation('relu', name=conv_name_base + '2a_relu')(x)

    x = ZeroPadding2D((1, 1), name=conv_name_base + '2b_zeropadding')(x)
    x = Conv2D(nb_filter2, (kernel_size, kernel_size),
                      name=conv_name_base + '2b', use_bias=False, trainable=trainable)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2b')(x)
    x = Activation('relu', name=conv_name_base + '2b_relu')(x)

    x = Conv2D(nb_filter3, (1, 1), name=conv_name_base + '2c', use_bias=False, trainable=trainable)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2c')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2c')(x)

    shortcut = Conv2D(nb_filter3, (1, 1), strides=strides,
                             name=conv_name_base + '1', use_bias=False, trainable=trainable)(input_tensor)
    shortcut = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '1')(shortcut)
    shortcut = Scale(axis=bn_axis, name=scale_name_base + '1')(shortcut)

    x = add([x, shortcut], name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x

def resnet152_model(n_classes, weights_path=None):
    '''Instantiate the ResNet152 architecture,
    # Arguments
        weights_path: path to pretrained weight file
    # Returns
        A Keras model instance.
    '''
    eps = 1.1e-5

    # Handle Dimension Ordering for different backends
    global bn_axis
    if K.image_dim_ordering() == 'tf':
        bn_axis = 3
        img_input = Input(shape=(224, 224, 3), name='data')
    else:
        bn_axis = 1
        img_input = Input(shape=(3, 224, 224), name='data')
            
    x = ZeroPadding2D((3, 3), name='conv1_zeropadding')(img_input)
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', use_bias=False, trainable=trainable)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name='bn_conv1')(x)
    x = Scale(axis=bn_axis, name='scale_conv1')(x)
    x = Activation('relu', name='conv1_relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='pool1')(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    for i in range(1,8):
        x = identity_block(x, 3, [128, 128, 512], stage=3, block='b'+str(i))

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    for i in range(1,36):
        x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b'+str(i))

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    #for classification
    #x_fc = AveragePooling2D((7, 7), name='avg_pool')(x)
    #x_fc = Flatten()(x_fc)
    #x_fc = Dense(1000, activation='softmax', name='fc1000')(x_fc)

    #new section for segmentation
    up1 = UpSampling2D((2,2))(x) #7 -> 14
    #up1 = ZeroPadding2D(((0,2),(0,2)))(up1) #160->162
    #concat1 = Concatenate(axis=3)([up7,add3])
    iden1 = Conv2D(smallest_layer*4, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(up1)#concat1
    conv1 = BatchNormalization()(up1)#concat1
    conv1 = Activation('relu')(conv1)
    conv1 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv1) #200
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv1) #200
    add1 = Add()([iden1,conv1])
    
    up2 = UpSampling2D((2,2))(add1) #14 -> 28
    #up2 = ZeroPadding2D(((0,2),(0,2)))(up2) #160->162
    #concat1 = Concatenate(axis=3)([up7,add3])
    iden2 = Conv2D(smallest_layer*4, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(up2)#concat2
    conv2 = BatchNormalization()(up2)#concat2
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv2) #200
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv2) #200
    add2 = Add()([iden2,conv2])
    
    up3 = UpSampling2D((2,2))(add2) #28 -> 56
    #up2 = ZeroPadding2D(((0,2),(0,2)))(up2) #160->162
    #concat1 = Concatenate(axis=3)([up7,add3])
    iden3 = Conv2D(smallest_layer*4, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(up3)#concat3
    conv3 = BatchNormalization()(up3)#concat3
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv3) #200
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv3) #200
    add3 = Add()([iden3,conv3])
    
    up4 = UpSampling2D((2,2))(add3) #56 -> 128
    #up2 = ZeroPadding2D(((0,2),(0,2)))(up2) #160->162
    #concat1 = Concatenate(axis=3)([up7,add3])
    iden4 = Conv2D(smallest_layer*4, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(up4)#concat3
    conv4 = BatchNormalization()(up4)#concat3
    conv4 = Activation('relu')(conv4)
    conv4 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv4) #200
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    conv4 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv4) #200
    add4 = Add()([iden4,conv4])
    
    up5 = UpSampling2D((2,2))(add4) #128 -> 224
    #up2 = ZeroPadding2D(((0,2),(0,2)))(up2) #160->162
    #concat1 = Concatenate(axis=3)([up7,add3])
    iden5 = Conv2D(smallest_layer*4, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(up5)#concat3
    conv5 = BatchNormalization()(up5)#concat3
    conv5 = Activation('relu')(conv5)
    conv5 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv5) #200
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)
    conv5 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv5) #200
    add5 = Add()([iden5,conv5])
    
    conv6 = Conv2D(n_classes, 3, activation ='sigmoid', padding = 'same', kernel_initializer = 'he_normal')(add5)
    
    model = Model(img_input, conv6)
    
     # load weights
    if weights_path:
        model.load_weights(weights_path, by_name=True)

    return model

In [ ]:
#modified Resnet152 again (?)
import cv2
import numpy as np
import copy

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Flatten, Activation, add
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import initializers
from keras.engine import Layer, InputSpec
from keras import backend as K

import sys
sys.setrecursionlimit(3000)

trainable = 'False'

class Scale(Layer):
    '''Custom Layer for ResNet used for BatchNormalization.
    
    Learns a set of weights and biases used for scaling the input data.
    the output consists simply in an element-wise multiplication of the input
    and a sum of a set of constants:
        out = in * gamma + beta,
    where 'gamma' and 'beta' are the weights and biases larned.
    # Arguments
        axis: integer, axis along which to normalize in mode 0. For instance,
            if your input tensor has shape (samples, channels, rows, cols),
            set axis to 1 to normalize per feature map (channels axis).
        momentum: momentum in the computation of the
            exponential average of the mean and standard deviation
            of the data, for feature-wise normalization.
        weights: Initialization weights.
            List of 2 Numpy arrays, with shapes:
            `[(input_shape,), (input_shape,)]`
        beta_init: name of initialization function for shift parameter
            (see [initializers](../initializers.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
        gamma_init: name of initialization function for scale parameter (see
            [initializers](../initializers.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
    '''
    def __init__(self, weights=None, axis=-1, momentum = 0.9, beta_init='zero', gamma_init='one', **kwargs):
        self.momentum = momentum
        self.axis = axis
        self.beta_init = initializers.get(beta_init)
        self.gamma_init = initializers.get(gamma_init)
        self.initial_weights = weights
        super(Scale, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        shape = (int(input_shape[self.axis]),)

        self.gamma = K.variable(self.gamma_init(shape), name='%s_gamma'%self.name)
        self.beta = K.variable(self.beta_init(shape), name='%s_beta'%self.name)
        self.trainable_weights = [self.gamma, self.beta]

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights

    def call(self, x, mask=None):
        input_shape = self.input_spec[0].shape
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]

        out = K.reshape(self.gamma, broadcast_shape) * x + K.reshape(self.beta, broadcast_shape)
        return out

    def get_config(self):
        config = {"momentum": self.momentum, "axis": self.axis}
        base_config = super(Scale, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

def identity_block(input_tensor, kernel_size, filters, stage, block):
    '''The identity_block is the block that has no conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    '''
    eps = 1.1e-5
    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    x = Conv2D(nb_filter1, (1, 1), name=conv_name_base + '2a', use_bias=False, trainable=trainable)(input_tensor)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2a')(x)
    x = Activation('relu', name=conv_name_base + '2a_relu')(x)

    x = ZeroPadding2D((1, 1), name=conv_name_base + '2b_zeropadding')(x)
    x = Conv2D(nb_filter2, (kernel_size, kernel_size), name=conv_name_base + '2b', use_bias=False, trainable=trainable)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2b')(x)
    x = Activation('relu', name=conv_name_base + '2b_relu')(x)

    x = Conv2D(nb_filter3, (1, 1), name=conv_name_base + '2c', use_bias=False, trainable=trainable)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2c')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2c')(x)

    x = add([x, input_tensor], name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x

def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    '''conv_block is the block that has a conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    Note that from stage 3, the first conv layer at main path is with subsample=(2,2)
    And the shortcut should have subsample=(2,2) as well
    '''
    eps = 1.1e-5
    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    x = Conv2D(nb_filter1, (1, 1), strides=strides, name=conv_name_base + '2a', use_bias=False, trainable=trainable)(input_tensor)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2a')(x)
    x = Activation('relu', name=conv_name_base + '2a_relu')(x)

    x = ZeroPadding2D((1, 1), name=conv_name_base + '2b_zeropadding')(x)
    x = Conv2D(nb_filter2, (kernel_size, kernel_size),
                      name=conv_name_base + '2b', use_bias=False, trainable=trainable)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2b')(x)
    x = Activation('relu', name=conv_name_base + '2b_relu')(x)

    x = Conv2D(nb_filter3, (1, 1), name=conv_name_base + '2c', use_bias=False, trainable=trainable)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2c')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2c')(x)

    shortcut = Conv2D(nb_filter3, (1, 1), strides=strides,
                             name=conv_name_base + '1', use_bias=False, trainable=trainable)(input_tensor)
    shortcut = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '1')(shortcut)
    shortcut = Scale(axis=bn_axis, name=scale_name_base + '1')(shortcut)

    x = add([x, shortcut], name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x

def resnet152_model(n_classes, weights_path=None):
    '''Instantiate the ResNet152 architecture,
    # Arguments
        weights_path: path to pretrained weight file
    # Returns
        A Keras model instance.
    '''
    eps = 1.1e-5

    # Handle Dimension Ordering for different backends
    global bn_axis
    if K.image_dim_ordering() == 'tf':
        bn_axis = 3
        img_input = Input(shape=(224, 224, 3), name='data')
    else:
        bn_axis = 1
        img_input = Input(shape=(3, 224, 224), name='data')
            
    x = ZeroPadding2D((3, 3), name='conv1_zeropadding')(img_input)
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', use_bias=False, trainable=trainable)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name='bn_conv1')(x)
    x = Scale(axis=bn_axis, name='scale_conv1')(x)
    x = Activation('relu', name='conv1_relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='pool1')(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    for i in range(1,8):
        x = identity_block(x, 3, [128, 128, 512], stage=3, block='b'+str(i))

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    for i in range(1,36):
        x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b'+str(i))

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    #for classification
    #x_fc = AveragePooling2D((7, 7), name='avg_pool')(x)
    #x_fc = Flatten()(x_fc)
    #x_fc = Dense(1000, activation='softmax', name='fc1000')(x_fc)

    #new section for segmentation
    up1 = UpSampling2D((2,2))(x) #7 -> 14
    #up1 = ZeroPadding2D(((0,2),(0,2)))(up1) #160->162
    #concat1 = Concatenate(axis=3)([up7,add3])
    iden1 = Conv2D(smallest_layer*4, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(up1)#concat1
    conv1 = BatchNormalization()(up1)#concat1
    conv1 = Activation('relu')(conv1)
    conv1 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv1) #200
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv1) #200
    add1 = Add()([iden1,conv1])
    
    up2 = UpSampling2D((4,4))(add1) #14 -> 56
    #up2 = ZeroPadding2D(((0,2),(0,2)))(up2) #160->162
    #concat1 = Concatenate(axis=3)([up7,add3])
    iden2 = Conv2D(smallest_layer*4, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(up2)#concat2
    conv2 = BatchNormalization()(up2)#concat2
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv2) #200
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv2) #200
    add2 = Add()([iden2,conv2])
    
    up3 = UpSampling2D((4,4))(add2) #56 -> 224
    #up2 = ZeroPadding2D(((0,2),(0,2)))(up2) #160->162
    #concat1 = Concatenate(axis=3)([up7,add3])
    iden3 = Conv2D(smallest_layer*4, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(up3)#concat3
    conv3 = BatchNormalization()(up3)#concat3
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv3) #200
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv3) #200
    add3 = Add()([iden3,conv3])
    
    conv4 = Conv2D(n_classes, 3, activation ='sigmoid', padding = 'same', kernel_initializer = 'he_normal')(add3)
    
    model = Model(img_input, conv4)
    
     # load weights
    if weights_path:
        model.load_weights(weights_path)#, by_name=True)

    return model

In [ ]:
#define dice coefficient
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)


In [ ]:
"""
A weighted version of categorical_crossentropy for keras (2.0.6). This lets you apply a weight to unbalanced classes.
@url: https://gist.github.com/wassname/ce364fddfc8a025bfab4348cf5de852d
@author: wassname
"""
from keras import backend as K
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss


In [ ]:
#trying lots of loss functions
def create_weighted_binary_crossentropy(zero_weight, one_weight):

    def weighted_binary_crossentropy(y_true, y_pred):

        # Original binary crossentropy (see losses.py):
        # K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1)

        # Calculate the binary crossentropy
        b_ce = K.binary_crossentropy(y_true, y_pred)

        # Apply the weights
        weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
        weighted_b_ce = weight_vector * b_ce

        # Return the mean error
        return K.mean(weighted_b_ce)

    return weighted_binary_crossentropy

In [ ]:
def jaccard_distance(y_true, y_pred, smooth=100):
    """Jaccard distance for semantic segmentation.
    Also known as the intersection-over-union loss.
    This loss is useful when you have unbalanced numbers of pixels within an image
    because it gives all classes equal weight. However, it is not the defacto
    standard for image segmentation.
    For example, assume you are trying to predict if
    each pixel is cat, dog, or background.
    You have 80% background pixels, 10% dog, and 10% cat.
    If the model predicts 100% background
    should it be be 80% right (as with categorical cross entropy)
    or 30% (with this loss)?
    The loss has been modified to have a smooth gradient as it converges on zero.
    This has been shifted so it converges on 0 and is smoothed to avoid exploding
    or disappearing gradient.
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
    # Arguments
        y_true: The ground truth tensor.
        y_pred: The predicted tensor
        smooth: Smoothing factor. Default is 100.
    # Returns
        The Jaccard distance between the two tensors.
    # References
        - [What is a good evaluation measure for semantic segmentation?](
           http://www.bmva.org/bmvc/2013/Papers/paper0032/paper0032.pdf)
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

In [ ]:
#only need 1 dimension for labels. reserve 25 images as a test set
labelstemp=resized_labels[:-100,...,1].reshape([labels.shape[0]-100,224,224,1])
datasettemp=resized_dataset[:-100,...].reshape([labels.shape[0]-100,224,224,3])
datasetnice=resized_dataset[:-100,...].reshape([labels.shape[0]-100,224,224,3])
#labelstemp=np.sum(datasettemp, axis=3) <= np.mean(np.sum(datasettemp, axis=3), axis=(1,2))[:,None,None]/2
#labelstemp=labelstemp.reshape([labels.shape[0]-25,224,224,1])
#datasettemp=np.sum(datasettemp, axis=3) >= np.mean(np.sum(datasettemp, axis=3), axis=(1,2))[:,None,None]/2
#datasettemp=datasettemp.reshape([labels.shape[0]-25,648,648,1])

#add all 4 rotations of 90 degrees to augment data
labelstrain=np.copy(labelstemp) #arrays used to train model
datasettrain=np.copy(datasettemp)
for i in range(3):
    labelstrain=np.append(labelstrain, np.rot90(labelstemp, k=i+1, axes=(1,2)), axis=0)
    datasettrain=np.append(datasettrain, np.rot90(datasettemp, k=i+1, axes=(1,2)), axis=0)

print(datasettrain.shape)
print(labelstrain.shape)
f = plt.figure()
image=573
f.add_subplot(1,2, 1)
plt.imshow(datasettrain[image,:,:,:].astype('float32')/2+[.5])
f.add_subplot(1,2, 2)
plt.imshow(labelstrain[image,:,:,0])
plt.show(block=True)

In [ ]:
#training!
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2,3,4,5,6,7,8"
no_gpus=8
smallest_layer=32
if no_gpus > 1:
    modelrough=multi_gpu_model(resnet152_model(1, weights_path='/home/ubuntu/MPhys-Project/model_weights/rough_model_weights.h5'), gpus=no_gpus)
#model=RESUNET(dataset.shape[1:],2)
else:
    modelrough=resnet152_model(1, weights_path='/home/ubuntu/MPhys-Project/model_weights/rough_model_weights.h5')
#model=multi_gpu_model(model,gpus=8)

from keras.metrics import categorical_accuracy
optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
#optimizer=rmsprop(lr=0.0001)
#loss=create_weighted_binary_crossentropy(0.01,0.99)
#loss=jaccard_distance
loss=dice_coef_loss
#loss='binary_crossentropy'
modelrough.compile(optimizer=optimizer,
                         loss=loss, metrics=[dice_coef, 'accuracy', 'binary_crossentropy'])
print(modelrough.summary())
#model.load_weights('/home/ubuntu/MPhys-Project/model_weights/rough_model_weights.h5')
#plot_model(model, to_file='model.png')

In [ ]:
#create the refined mask model and load the weights
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2,3,4,5,6,7,8"
no_gpus=1
smallest_layer=32
if no_gpus > 1:
    refined_mask_model=multi_gpu_model(RESUNETMEDIUM((224,224,3), 1), gpus=no_gpus)
else:
    refined_mask_model=RESUNETMEDIUM((224,224,3), 1)

from keras.metrics import categorical_accuracy
optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
loss=dice_coef_loss
refined_mask_model.compile(optimizer=optimizer,
                         loss=loss, metrics=[dice_coef, 'accuracy', 'binary_crossentropy'])

refined_mask_model.load_weights('/home/ubuntu/MPhys-Project/model_weights/refine_model_weights.h5')


In [ ]:
modelrough.load_weights('all_data_best_weights.h5')
mask=modelrough.predict(resized_dataset, batch_size=8*8)

In [ ]:
#create the masks and dilate them, then apply them to the dataset

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (6, 6))
dilated_mask=np.zeros(mask.shape)
open_closed_masks = np.zeros((mask.shape), dtype='float32')
for i in range(mask.shape[0]):
    open_closed_masks[i,...,0] = cv2.morphologyEx(cv2.morphologyEx(mask[i,...,0].astype('float32'), cv2.MORPH_OPEN, kernel), cv2.MORPH_CLOSE, kernel)
    dilated_mask[i,...,0]=cv2.dilate(open_closed_masks[i,...,0], kernel, iterations=6)>=0.5
f = plt.figure(figsize=(18,18))
datasetmasked = np.zeros(resized_dataset.shape)
labelsmasked = resized_labels*dilated_mask
for i in range(3):
    datasetmasked[...,i]=(resized_dataset[...,i]+1)/2*dilated_mask[...,0]

image=45
f.add_subplot(1,6, 1)
plt.imshow(mask[image,:,:,0])
f.add_subplot(1,6, 2)
plt.imshow(dilated_mask[image,:,:,0])
f.add_subplot(1,6, 3)
plt.imshow(resized_dataset[image,:,:,:].astype('float32')/2+[.5])
f.add_subplot(1,6, 4)
plt.imshow(resized_labels[image,:,:,1])
f.add_subplot(1,6, 5)
plt.imshow(datasetmasked[image,:,:,:].astype('float32'))
f.add_subplot(1,6, 6)
plt.imshow(labelsmasked[image,:,:,1])
plt.show(block=True)

In [ ]:
#model.fit(x_train, y_train, batch_size=100, epochs=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1,) #min_lr=5e-7, verbose=1)
#earlystopper = EarlyStopping(patience=8, verbose=1)
checkpointer = ModelCheckpoint(filepath='Checkpoint_Weights.h5', save_best_only=True, monitor='val_acc', verbose=1)

batch_size=1*no_gpus
history = model.fit(dataset_threshold, tru, batch_size=batch_size, epochs=6, callbacks=[reduce_lr, checkpointer], validation_split=0.2)
#score = model.evaluate(x_test, y_test, batch_size=5)

In [ ]:
#Manual shuffle just in case
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

datasettrain, labelstrain=shuffle_in_unison(datasettrain,labelstrain)
print(datasettrain.shape[0])
print(labelstrain.shape[0])

In [ ]:
#model.load_weights('Best_Weights.h5')
#model.load_weights('resnet_152_smaller_best_weights.h5')
modelrefine.load_weights('modelrefine_weights_best.h5')

In [ ]:
f = plt.figure()
image=109
f.add_subplot(1,2, 1)
plt.imshow(datasettrain[image,:,:,:].astype('float32')/2+[.5])
f.add_subplot(1,2, 2)
plt.imshow(labelstrain[image,:,:,0])
plt.show(block=True)

In [ ]:
#Training with data augmentation actually implemented properly. This was not used in the end.

model.load_weights('all_data_best_weights_rotations.h5')
batch_size_per_gpu=6
batch_size=batch_size_per_gpu*no_gpus
validation_split=0.2

data=datasettrain
truth=labelstrain

epochs=50

datagen = ImageDataGenerator(
    #rotation_range=15,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #shear_range=0.2,
    #horizontal_flip=True,
    validation_split=validation_split)



# we create two instances with the same arguments
data_gen_args = dict(
                    rotation_range=45,
                    #width_shift_range=0.2,
                    #height_shift_range=0.2,
                    #shear_range=0.2,
                    horizontal_flip=True,
                    zoom_range=0.2,)
                    #validation_split=validation_split)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
image_datagen.fit(data, augment=True, seed=seed)
mask_datagen.fit(truth, augment=True, seed=seed)

image_generator_train = image_datagen.flow(data, batch_size=batch_size, shuffle=True, seed=seed,)# subset='training')
mask_generator_train = image_datagen.flow(truth, batch_size=batch_size, shuffle=True, seed=seed,)# subset='training')
#image_generator_validate = image_datagen.flow(data, batch_size=batch_size, shuffle=True, seed=seed, subset='validation')
#mask_generator_validate = image_datagen.flow(truth, batch_size=batch_size, shuffle=True, seed=seed, subset='validation')

# combine generators into one which yields image and masks
train_generator = zip(image_generator_train, mask_generator_train)
#validation_generator = zip(image_generator_validate, mask_generator_validate)

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=6, min_lr=5e-9, verbose=1)
#earlystopper = EarlyStopping(patience=20, verbose=1, monitor='val_loss')
checkpointer = ModelCheckpoint(filepath='model_weights_full_data.h5', save_best_only=True, monitor='loss', verbose=1)

if no_gpus == 1:
    history = model.fit_generator(train_generator,
                    epochs=epochs, callbacks=[reduce_lr, checkpointer], steps_per_epoch=data.shape[0]//batch_size,)
                    #validation_data=validation_generator,
                    #validation_steps=data.shape[0]*validation_split//batch_size,)#, use_multiprocessing=True)
else:
    history = model.fit_generator(train_generator,
                    epochs=epochs, callbacks=[reduce_lr, checkpointer], steps_per_epoch=data.shape[0]//batch_size,)
                    #validation_data=validation_generator,
                    #validation_steps=data.shape[0]*validation_split//batch_size_per_gpu, use_multiprocessing=True)

In [ ]:
data.shape

In [ ]:
epochs = np.arange(len(history.history['loss']))
plt.plot(epochs, history.history['loss'], label='loss')
#plt.plot(epochs, history.history['val_loss'], label='val_loss')
#plt.plot(epochs, history.history['dice_coef'], label='dice_coef')
#plt.plot(epochs, history.history['val_dice_coef'], label='val_dice_coef')
plt.plot(epochs, history.history['acc'], label='accuracy')
#plt.plot(epochs, history.history['val_acc'], label='val_accuracy')
plt.legend(loc='upper left')
plt.show()
print(model.summary())



In [ ]:
plot_model(model, to_file='model.png')

In [ ]:
#model.save("resnet152_smaller.h5")
model.save_weights("rough_model_all_data.h5")

In [ ]:
np.save(history, 'training_history.npy')

In [ ]:
#model.load_weights('all_data_best_weights_rotations_longer_training.h5')
#Plot some examples
no_gpus = 1
f = plt.figure(figsize=(18,300))
samples=datasetmasked[resized_labels[...,0].any(axis=(1,2)),...][0:100:5,:,:,:].astype('float32').reshape(20,224,224,3)
original_images=resized_dataset[resized_labels[...,0].any(axis=(1,2)),...][0:100:5,:,:,:].astype('float32').reshape(20,224,224,3)
annotations=resized_labels[resized_labels[...,0].any(axis=(1,2)),...,0][0:100:5,:,:]
prediction=refined_mask_model.predict(samples, batch_size=no_gpus*8)
for image, sample in enumerate(samples):
    f.add_subplot(samples.shape[0],5, 5*image+1)
    plt.imshow(np.clip(original_images[image,:,:,:]/2+0.5, 0, 1))
    f.add_subplot(samples.shape[0],5, 5*image+2)
    plt.imshow(np.clip(samples[image,:,:,:], 0, 1))
    f.add_subplot(samples.shape[0],5, 5*image+3)
    plt.imshow(annotations[image,:,:])
    f.add_subplot(samples.shape[0],5, 5*image+4)
    plt.imshow(prediction[image,:,:,0]>=0.5)
    f.add_subplot(samples.shape[0],5, 5*image+5)
    plt.imshow(open_closed_masks[image,:,:,0]>=0.5)
f.tight_layout()
f.subplots_adjust(hspace=0.1)
plt.show(block=True)

In [ ]:
dataset=2*dataset.astype('float16')-1


In [ ]:
#Model metrics

#model.load_weights('all_data_best_weights.h5')
pre = refined_mask_model.predict(datasetmasked[-100:,...], batch_size=8*no_gpus)>=0.5
print(pre.shape)
tru = resized_labels[-100:,...,1].reshape(100,224,224,1).astype('bool')

true_positives = tru*pre
false_positives = pre*~tru
true_negatives = ~tru*~pre
false_negatives = tru*~pre

tot_tp = np.sum(true_positives)
tot_fp = np.sum(false_positives)
tot_tn = np.sum(true_negatives)
tot_fn = np.sum(false_negatives)

precision = tot_tp/(tot_tp+tot_fp)
recall = tot_tp/(tot_tp+tot_fn)
f1 = 2*precision*recall/(precision + recall)
TPR = recall #recall is the same as true positive rate
TNR = tot_tn/(tot_tn+tot_fp) #true negative rate
bACC = (TPR+TNR)/2 #Balanced accuracy score

print('precision = ', precision)
print('recall = ', recall)
print('f1 score = ', f1)
print('true positive rate = ', TPR)
print('true negative rate = ', TNR)
print('balanced accuracy score = ', bACC)

pre = pre[...,0].any(axis=(1,2))
tru = tru[..., 0].any(axis=(1,2))

true_positives = tru*pre
false_positives = pre*~tru
true_negatives = ~tru*~pre
false_negatives = tru*~pre

tot_tp = np.sum(true_positives)
tot_fp = np.sum(false_positives)
tot_tn = np.sum(true_negatives)
tot_fn = np.sum(false_negatives)

precision = tot_tp/(tot_tp+tot_fp)
recall = tot_tp/(tot_tp+tot_fn)
f1 = 2*precision*recall/(precision + recall)
TPR = recall #recall is the same as true positive rate
TNR = tot_tn/(tot_tn+tot_fp) #true negative rate
bACC = (TPR+TNR)/2 #Balanced accuracy score

print('image precision = ', precision)
print('image recall = ', recall)
print('image f1 score = ', f1)
print('image true positive rate = ', TPR)
print('image true negative rate = ', TNR)
print('image balanced accuracy score = ', bACC)

In [ ]:
tru.shape

In [ ]:
from sklearn.metrics import classification_report
y_true = labelsnew.tolist()
y_pred = model.predict(dataset, batch_size=batch_size)
target_names = ['not POC', 'POC']
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
a=np.eye(3).reshape((3,3,1))
b=np.copy(a)
for i in range(3):
    b=np.append(b, np.rot90(a, k=i+1), axis=2)
    print(i)
print(b[:,:,])
print(a[:,:,0])
print(b.shape)



In [ ]:
pre = np.array([True, True, False, False])
tru = np.array([False, True, True, False])

print('tp = ', tru*pre)
print('fp = ', pre*~tru)
print('tn = ', ~tru * ~pre)
print('fn = ', tru * ~ pre)
      

In [ ]:
a=np.array([[1,2,3],[4,5,6],[7,8,9]])
a.resize(2,2)
a

In [ ]:
print(original_images)

In [ ]:
np.amax(resized_dataset)

In [ ]:
resized_dataset.shape



In [ ]:
datasetmasked.shape
